In [4]:
# 필요한 패키지 설치
!pip install pandas torch transformers openpyxl



In [7]:
!pip uninstall pandas
!pip install pandas

Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/pandas-2.2.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/pandas/*
Proceed (Y/n)? y
  Successfully uninstalled pandas-2.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 77.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.6.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [9]:
# 필요한 라이브러리 임포트
# import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
import random


In [10]:
# # 엑셀 파일 불러오기
# file_path = './hall_dataset_preprocessing_final.xlsx'  # 파일 경로
# df = pd.read_excel(file_path)

# # 컬럼 이름 확인
# print(df.columns)

!pip install datasets

from datasets import load_dataset
ds = load_dataset("mteb/amazon_reviews_multi", "all_languages")

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00


AttributeError: partially initialized module 'pandas' has no attribute '_pandas_parser_CAPI' (most likely due to a circular import)

In [110]:
print(ds['train'].column_names)

['id', 'text', 'label', 'label_text']


In [112]:
# 데이터프레임에서 필요한 정보 추출 (예시로 베뉴 이름, 가격, 키워드, 단점 등을 사용)
venue_data = {}
for row in ds['train']:  # 'train' 스플릿에서 데이터 추출
    venue_data[row['id']] = {  # 'review_id'는 적절한 id 컬럼으로 수정
        "text": row['text']  # 'review_body'가 텍스트에 해당하는 컬럼명으로 수정
    }

user_sequences = {
    'user1': ["de_0032251"],
    'user2': ["de_0466272"],
    # 'user3': ["브라이튼 하우스 한남", "S가든 웨딩홀 대전", "엘타워 그레이스홀"],
}

KeyboardInterrupt: 

In [99]:
# BERT 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class BERT4RecDataset(Dataset):
    def __init__(self, user_sequences, venue_data, max_seq_len=5, mask_prob=0.15):
        self.user_sequences = user_sequences
        self.venue_data = venue_data
        self.max_seq_len = max_seq_len
        self.mask_prob = mask_prob
        self.venue_names = list(venue_data.keys())
        self.vocab_size = len(self.venue_names)
        self.data = self.prepare_data()

    def prepare_data(self):
        data = []
        for seq in self.user_sequences.values():
            indexed_seq = []
            for item in seq:
                if item in self.venue_names:
                    indexed_seq.append(self.venue_names.index(item))
                else:
                    print(f"Warning: {item} is not in venue_names, replacing with [UNK]")
                    indexed_seq.append(self.vocab_size - 1)  # [UNK] 토큰으로 대체

            # 패딩 또는 잘라내기
            if len(indexed_seq) < self.max_seq_len:
                indexed_seq = [0] * (self.max_seq_len - len(indexed_seq)) + indexed_seq  # 패딩
            else:
                indexed_seq = indexed_seq[:self.max_seq_len]
            data.append(indexed_seq)
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        seq = self.data[idx]
        input_ids = torch.tensor(seq, dtype=torch.long)
        labels = torch.tensor(seq, dtype=torch.long)
        masked_input = input_ids.clone()
        mask = torch.zeros_like(input_ids, dtype=torch.bool)

        # 마스킹 로직
        for i in range(len(seq)):
            if random.random() < self.mask_prob:
                mask[i] = True
                prob = random.random()
                if prob < 0.8:
                    masked_input[i] = self.vocab_size - 1  # 마스킹 토큰 추가
                elif prob < 0.9:
                    masked_input[i] = random.choice(list(range(self.vocab_size)))  # 랜덤하게 다른 토큰 선택

        return masked_input, labels, mask


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [100]:
# BERT4Rec 모델 정의 (간단화된 BERT 구조)
class BERT4RecModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=64, num_heads=4, num_layers=2, max_seq_len=5, dropout=0.1):
        super(BERT4RecModel, self).__init__()
        self.token_embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.position_embedding = nn.Embedding(max_seq_len, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embed_dim, vocab_size)

    def forward(self, input_ids):
        positions = torch.arange(0, input_ids.size(1)).unsqueeze(0).to(input_ids.device)
        x = self.token_embedding(input_ids) + self.position_embedding(positions)
        x = x.transpose(0, 1)
        x = self.transformer_encoder(x)
        x = x.transpose(0, 1)
        logits = self.fc(x)
        return logits


In [101]:
# 데이터셋 로드 및 DataLoader 설정
dataset = BERT4RecDataset(user_sequences, venue_data)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# 모델 초기화
vocab_size = len(dataset.venue_names)
model = BERT4RecModel(vocab_size=vocab_size)

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# 모델 학습
model.train()
for epoch in range(15):  # 예시로 10번의 epoch
    total_loss = 0
    for batch in dataloader:
        input_ids, labels, mask = batch
        optimizer.zero_grad()
        logits = model(input_ids)
        logits = logits.view(-1, vocab_size)
        labels = labels.view(-1)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader):.4f}")


Epoch 1, Loss: 5.0041
Epoch 2, Loss: 3.0385
Epoch 3, Loss: 2.2324
Epoch 4, Loss: 1.6245
Epoch 5, Loss: 1.5177
Epoch 6, Loss: 2.0615
Epoch 7, Loss: 1.7183
Epoch 8, Loss: 1.1170
Epoch 9, Loss: 1.0133
Epoch 10, Loss: 1.0203
Epoch 11, Loss: 0.8564
Epoch 12, Loss: 0.8335
Epoch 13, Loss: 0.7277
Epoch 14, Loss: 0.7142
Epoch 15, Loss: 0.6738


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [102]:
def recommend_venues(model, user_sequence, dataset, top_k=5):
    # user_sequence를 인덱스로 변환
    input_ids = []
    for venue in user_sequence:
        if venue in dataset.venue_names:
            input_ids.append(dataset.venue_names.index(venue))
        else:
            input_ids.append(dataset.vocab_size - 1)  # 없는 경우 [UNK] 토큰 사용

    # 패딩 처리 (시퀀스 길이를 모델의 입력 크기에 맞추기)
    max_seq_len = dataset.max_seq_len
    if len(input_ids) < max_seq_len:
        input_ids = [0] * (max_seq_len - len(input_ids)) + input_ids  # 앞쪽에 패딩 추가
    else:
        input_ids = input_ids[-max_seq_len:]  # 길면 자르기

    input_ids = torch.tensor([input_ids], dtype=torch.long)  # 배치 차원 추가

    # 모델을 통해 예측
    model.eval()  # 평가 모드로 전환
    with torch.no_grad():
        logits = model(input_ids)
        predictions = logits[0, -1, :]  # 마지막 토큰에 대한 예측 결과 추출

    # 상위 top_k개의 추천 웨딩홀 선택
    top_k_indices = torch.topk(predictions, top_k).indices.tolist()
    recommended_venues = [dataset.venue_names[idx] for idx in top_k_indices]

    return recommended_venues


In [103]:
def recommend_venues_for_all_users(model, user_sequences, dataset, top_k=5):
    recommendations = {}
    for user, sequence in user_sequences.items():
        recommended = recommend_venues(model, sequence, dataset, top_k)
        recommendations[user] = recommended
    return recommendations


In [104]:
# 모든 사용자에게 추천 생성
recommendations = recommend_venues_for_all_users(model, user_sequences, dataset)

# 결과 출력
for user, rec in recommendations.items():
    print(f"{user}에게 추천된 웨딩홀: {rec}")


user1에게 추천된 웨딩홀: ['휘겸재', '판교W스퀘어', '롯데호텔 소공동', '더채플앳 논현', '엘리애나 호텔 3F 임페리얼홀']
user2에게 추천된 웨딩홀: ['T웨딩홀', '소피텔 앰배서더 서울 그랜드볼룸', '크레스트72 가든홀 ', '엘블레스 양재', '브라이튼 하우스']
